# Spark titanic Machine Learning

## 요약
titanic 데이터를 활용해 머신러닝을 구현
- EDA에서 null값 처리, 칼럼 추가/삭제 작업
- feature 라이브러리에서 StringIndexer, VectorAssembler 활용
- train, test 데이터 구분해서 비교

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

conf  = SparkConf().setMaster('local').setAppName('spark_ml')
spark = SparkContext(conf=conf)

sqlCtx = SQLContext(spark)
sqlCtx

In [2]:
titanic = sqlCtx.read.csv('./data/spark_titanic_train.csv',
                         header=True,
                         inferSchema=True)
type(titanic)

pyspark.sql.dataframe.DataFrame

In [3]:
titanic.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [4]:
titanic.show()
# 결과를 보고 전처리가 필요함을 알아야 한다.

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [5]:
titanic.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [6]:
titanic.select(['Survived', 'Pclass', 'Embarked']).show()

+--------+------+--------+
|Survived|Pclass|Embarked|
+--------+------+--------+
|       0|     3|       S|
|       1|     1|       C|
|       1|     3|       S|
|       1|     1|       S|
|       0|     3|       S|
|       0|     3|       Q|
|       0|     1|       S|
|       0|     3|       S|
|       1|     3|       S|
|       1|     2|       C|
|       1|     3|       S|
|       1|     1|       S|
|       0|     3|       S|
|       0|     3|       S|
|       0|     3|       S|
|       1|     2|       S|
|       0|     3|       Q|
|       1|     2|       S|
|       0|     3|       S|
|       1|     3|       C|
+--------+------+--------+
only showing top 20 rows



### EDA

In [7]:
# 생존자 수 count
titanic.groupBy('Survived').count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



In [8]:
# 선실 정보와 생존 여부에 따른 count
titanic.groupBy('Pclass', 'Survived').count().show()

+------+--------+-----+
|Pclass|Survived|count|
+------+--------+-----+
|     1|       0|   80|
|     3|       1|  119|
|     1|       1|  136|
|     2|       1|   87|
|     2|       0|   97|
|     3|       0|  372|
+------+--------+-----+



In [9]:
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit

In [10]:
# This function use to print feature with null values and null count 
def null_value_count(df):
  null_columns_counts = []
  numRows = df.count()
  for k in df.columns:
    nullRows = df.where(col(k).isNull()).count()
    if(nullRows > 0):
      temp = k,nullRows
      null_columns_counts.append(temp)
  return(null_columns_counts)

In [11]:
null_list = null_value_count(titanic)

In [12]:
null_list

[('Age', 177), ('Cabin', 687), ('Embarked', 2)]

In [13]:
sqlCtx.createDataFrame(null_list, ['column', 'cnt']).show()

+--------+---+
|  column|cnt|
+--------+---+
|     Age|177|
|   Cabin|687|
|Embarked|  2|
+--------+---+



In [14]:
# 나이 평균 구하기
titanic.select(mean('Age')).show()

+-----------------+
|         avg(Age)|
+-----------------+
|29.69911764705882|
+-----------------+



In [15]:
titanic.select('Name').show()

+--------------------+
|                Name|
+--------------------+
|Braund, Mr. Owen ...|
|Cumings, Mrs. Joh...|
|Heikkinen, Miss. ...|
|Futrelle, Mrs. Ja...|
|Allen, Mr. Willia...|
|    Moran, Mr. James|
|McCarthy, Mr. Tim...|
|Palsson, Master. ...|
|Johnson, Mrs. Osc...|
|Nasser, Mrs. Nich...|
|Sandstrom, Miss. ...|
|Bonnell, Miss. El...|
|Saundercock, Mr. ...|
|Andersson, Mr. An...|
|Vestrom, Miss. Hu...|
|Hewlett, Mrs. (Ma...|
|Rice, Master. Eugene|
|Williams, Mr. Cha...|
|Vander Planke, Mr...|
|Masselmani, Mrs. ...|
+--------------------+
only showing top 20 rows



In [16]:
display(type('Name'))
# regexp_extract(col('Name'))

titanic01 = titanic.withColumn( 'initial', regexp_extract(col('Name'), '([A-Za-z]+)\.', 1) )

str

In [17]:
titanic01.select('initial').distinct().show()

+--------+
| initial|
+--------+
|     Don|
|    Miss|
|Countess|
|     Col|
|     Rev|
|    Lady|
|  Master|
|     Mme|
|    Capt|
|      Mr|
|      Dr|
|     Mrs|
|     Sir|
|Jonkheer|
|    Mlle|
|   Major|
|      Ms|
+--------+



In [18]:
titanic01.groupby('initial').avg('age').collect()

[Row(initial='Don', avg(age)=40.0),
 Row(initial='Miss', avg(age)=21.773972602739725),
 Row(initial='Countess', avg(age)=33.0),
 Row(initial='Col', avg(age)=58.0),
 Row(initial='Rev', avg(age)=43.166666666666664),
 Row(initial='Lady', avg(age)=48.0),
 Row(initial='Master', avg(age)=4.574166666666667),
 Row(initial='Mme', avg(age)=24.0),
 Row(initial='Capt', avg(age)=70.0),
 Row(initial='Mr', avg(age)=32.368090452261306),
 Row(initial='Dr', avg(age)=42.0),
 Row(initial='Mrs', avg(age)=35.898148148148145),
 Row(initial='Sir', avg(age)=49.0),
 Row(initial='Jonkheer', avg(age)=38.0),
 Row(initial='Mlle', avg(age)=24.0),
 Row(initial='Major', avg(age)=48.5),
 Row(initial='Ms', avg(age)=28.0)]

In [19]:
titanic01 = titanic01.replace(['Mlle','Mme', 'Ms', 'Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
               ['Miss','Miss','Miss','Mr','Mr',  'Mrs',  'Mrs',  'Other',  'Other','Other','Mr','Mr','Mr'])

In [20]:
titanic01.filter(titanic01['Age'] == 48).select('initial').show()

+-------+
|initial|
+-------+
|     Mr|
|     Mr|
|    Mrs|
|     Mr|
|     Mr|
|    Mrs|
|    Mrs|
|     Mr|
|    Mrs|
+-------+



In [21]:
# null 처리 방법
titanic.groupby('Embarked').count().show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|   77|
|    null|    2|
|       C|  168|
|       S|  644|
+--------+-----+



In [22]:
titanic01 = titanic01.na.fill({'Embarked' : 'S'})

In [23]:
titanic01.groupby('Embarked').count().show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|   77|
|       C|  168|
|       S|  646|
+--------+-----+



In [24]:
# Cabin 칼럼 삭제
titanic01 = titanic01.drop('Cabin')

In [25]:
titanic01.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = false)
 |-- initial: string (nullable = true)



In [26]:
# 파생 컬럼 생성
titanic01 = titanic01.withColumn('Family_Size', col('SibSp') + col('Parch'))

In [27]:
titanic01.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = false)
 |-- initial: string (nullable = true)
 |-- Family_Size: integer (nullable = true)



In [28]:
titanic01.select('Family_Size').show()

+-----------+
|Family_Size|
+-----------+
|          1|
|          1|
|          0|
|          1|
|          0|
|          0|
|          0|
|          4|
|          2|
|          1|
|          2|
|          0|
|          0|
|          6|
|          0|
|          0|
|          5|
|          0|
|          1|
|          0|
+-----------+
only showing top 20 rows



In [29]:
titanic01.groupby('Family_Size').count().show()

+-----------+-----+
|Family_Size|count|
+-----------+-----+
|          1|  161|
|          6|   12|
|          3|   29|
|          5|   22|
|          4|   15|
|          7|    6|
|         10|    7|
|          2|  102|
|          0|  537|
+-----------+-----+



In [30]:
titanic01 = titanic01.withColumn('Alone', lit(0))

In [31]:
titanic01.select('Alone').show()

+-----+
|Alone|
+-----+
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
+-----+
only showing top 20 rows



In [32]:
titanic01 = titanic01.withColumn('Alone', when(titanic01['Family_Size'] == 0, 1).otherwise(titanic01['Alone']))

In [33]:
titanic01.select('Alone').show()

+-----+
|Alone|
+-----+
|    0|
|    0|
|    1|
|    0|
|    1|
|    1|
|    1|
|    0|
|    0|
|    0|
|    0|
|    1|
|    1|
|    0|
|    1|
|    1|
|    0|
|    1|
|    0|
|    1|
+-----+
only showing top 20 rows



In [36]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(titanic01) for column in ["Sex","Embarked","initial"]]
pipeline = Pipeline(stages=indexers)
titanic01 = pipeline.fit(titanic01).transform(titanic01)

In [37]:
titanic01.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = false)
 |-- initial: string (nullable = true)
 |-- Family_Size: integer (nullable = true)
 |-- Alone: integer (nullable = false)
 |-- Sex_index: double (nullable = false)
 |-- Embarked_index: double (nullable = false)
 |-- initial_index: double (nullable = false)



In [48]:
titanic01 = titanic01.na.fill({'Age' : 20})

In [49]:
titanic01 = titanic01.drop("PassengerId","Name","Ticket","Cabin","Embarked","Sex","initial")

In [50]:
titanic01.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Age: double (nullable = false)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Family_Size: integer (nullable = true)
 |-- Alone: integer (nullable = false)
 |-- Sex_index: double (nullable = false)
 |-- Embarked_index: double (nullable = false)
 |-- initial_index: double (nullable = false)



In [51]:
from pyspark.ml.feature import VectorAssembler

In [52]:
feature = VectorAssembler(inputCols = titanic01.columns[1:], outputCol = 'feature')
feature_vector = feature.transform(titanic01)

In [53]:
feature_vector.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Age: double (nullable = false)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Family_Size: integer (nullable = true)
 |-- Alone: integer (nullable = false)
 |-- Sex_index: double (nullable = false)
 |-- Embarked_index: double (nullable = false)
 |-- initial_index: double (nullable = false)
 |-- feature: vector (nullable = true)



In [54]:
trainData, testData = feature_vector.randomSplit([.8, .2], seed=100)

In [57]:
# type(trainData)
trainData.show()

+--------+------+----+-----+-----+--------+-----------+-----+---------+--------------+-------------+--------------------+
|Survived|Pclass| Age|SibSp|Parch|    Fare|Family_Size|Alone|Sex_index|Embarked_index|initial_index|             feature|
+--------+------+----+-----+-----+--------+-----------+-----+---------+--------------+-------------+--------------------+
|       0|     1| 2.0|    1|    2|  151.55|          3|    0|      1.0|           0.0|          1.0|[1.0,2.0,1.0,2.0,...|
|       0|     1|18.0|    1|    0|   108.9|          1|    0|      0.0|           1.0|          0.0|[1.0,18.0,1.0,0.0...|
|       0|     1|19.0|    1|    0|    53.1|          1|    0|      0.0|           0.0|          0.0|(10,[0,1,2,4,5],[...|
|       0|     1|19.0|    3|    2|   263.0|          5|    0|      0.0|           0.0|          0.0|[1.0,19.0,3.0,2.0...|
|       0|     1|20.0|    0|    0|     0.0|          0|    1|      0.0|           0.0|          0.0|(10,[0,1,6],[1.0,...|
|       0|     1|20.0|  

## 모델링
- SparkML (DTC, LR, RFC, GDTC, NB, SVM)

In [58]:
# LogisticRegression
# 데이터의 범주가 0, 1 사이의 값으로 예측하는 분류 (classification) 알고리즘

from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol = 'Survived', featuresCol = 'feature')
lr_model = lr.fit(trainData)
lr_pred  = lr_model.transform(testData)

In [60]:
# type(lr_pred)
# lr_pred.printSchema()
lr_pred.select('prediction', 'Survived', 'feature').show()

+----------+--------+--------------------+
|prediction|Survived|             feature|
+----------+--------+--------------------+
|       0.0|       0|(10,[0,1,6],[1.0,...|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       1.0|       0|(10,[0,1,4,6,8],[...|
|       1.0|       0|(10,[0,1,4,6,8],[...|
|       1.0|       0|[1.0,27.0,0.0,2.0...|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|(10,[0,1,6],[1.0,...|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|(10,[0,1,3,4,5],[...|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|[1.0,64.0,1.0,4.0...|
|       0.0|       0|(10,[0,1,4,6],[2....|
|       0.0|       0|(10,[0,1,4,6],[2....|
+----------

In [62]:
# 평가
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol= 'Survived',
                                             predictionCol = 'prediction',
                                             metricName = 'accuracy')

In [64]:
acc = evaluator.evaluate(lr_pred)
print('acc :', acc)
print('err :', 1.0-acc)

acc : 0.8166666666666667
err : 0.18333333333333335
